<a href="https://colab.research.google.com/github/Julianfried/Diplomatura-en-IA-Instituto-Humai/blob/main/Automatizacion%20y%20Web%20Scrapping/ejercicio/Introducci%C3%B3n_a_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios Scraping I

In [3]:
from bs4 import BeautifulSoup
import requests
import os
import re

## 1) Titulos de noticias
Descargar la pagina principal de noticias de Tecnología. Utilizar Regex para obtener los titulos de las noticias.

Ayuda: 
- los titulos estan encerrados por el tag `h2`
- No tiene que contenter el nombre de otra categoria

In [ ]:
infobae_url = 'https://www.infobae.com/tag/tecnologia/'

response = requests.get(infobae_url).text



In [ ]:
patron = r'<h2.*?>([^<>]+)</h2>'

titulos = re.findall(patron, response)

print(titulos)

['Los límites y desafíos de la inteligencia artificial', 'India, la nueva potencia global (II)', 'El Salvador aprobó una ley que elimina impuestos de renta, ganancias y propiedad a la industria tecnológica', 'Red 6G: qué beneficios y desventajas traerá esta tecnología', 'Tecnología OLED en televisores: qué atributos tienen los dispositivos que revolucionaron la forma de ver TV', 'La inteligencia artificial en la educación', 'Así reconocía Steve Jobs a los mejores trabajadores para Apple', 'Bennett Miller se pregunta si estamos listos para la Inteligencia Artificial', 'INVAP: Soberanía tecnológica y desarrollo nacional desde una visión pragmática  y sin subsidios estatales', 'Cómo la inteligencia artificial cambiará el futuro de las tareas domésticas', 'Startups e innovación', 'Comercio electrónico: cómo poner la tecnología al servicio de los usuarios', 'La jefa de ciberseguridad británica alertó sobre la amenaza de China a Occidente: “El desafío que define una época”', 'Los dilemas de 

## 2) Spinetta

Descargar letras de canciones

Utilizando beautiful soup descargar todas las canciones de [Spinetta](https://es.wikipedia.org/wiki/Luis_Alberto_Spinetta) que hay en [letras.com](https://www.letras.com/spinetta/)

In [ ]:
letras_url = "https://www.letras.com"

def descargar_letras(artista):
    dir_path = '/content/letras'

    path = os.path.join(dir_path, artista)

    if not os.path.exists(dir_path):
      os.mkdir(dir_path)
    
    if not os.path.exists(path):
      os.mkdir(path)
    
    os.chdir(path)


    link = os.path.join(letras_url, artista)
    codigo_html = requests.get(link).text
    soup = BeautifulSoup(codigo_html, 'html.parser')
    canciones = soup.find_all('a', class_ = 'song-name')

    for cancion in canciones:
      titulo = cancion.span.text
      url = letras_url + cancion['href']
      song_html = requests.get(url).text
      song_soup = BeautifulSoup(song_html)

      letra_html = song_soup.find('div', attrs={'class' : 'cnt-letra'})
      letra = ''

      for parrafo in letra_html.find_all('p'):
        texto = parrafo.get_text()

        letra += re.sub(r'([A-Z])', '\n\g<1>', texto)

        letra += '\n'
      
      with open (f'{titulo}.txt', 'w') as file:
        file.write(letra)
      
    os.chdir('/content')
      


    return 
        
artista = "luis-alberto-spinetta"
descargar_letras(artista)

## 3) Datos de la Formula 1

### 3.1) Obtener todos los campeones del mundo
Utilizando beautiful soup se pide descargar los siguientes datos:
- Año del campeonato
- Nombre del piloto
- Nombre del equipo
- Cantidad de carreras
- Cantidad de pilotos

In [ ]:
f1_piloto_url = 'https://pitwall.app/seasons?view=grid'




In [ ]:
pilotos = []
response = requests.get(f1_piloto_url)
soup = BeautifulSoup(response.text, 'html.parser')

grid_data = soup.find('div', id='seasons-grid')

for index, element in enumerate(grid_data.find_all('a')):
  anio = element.find('div', class_="top-label driver-number").text.strip()
  info = element.find('div', class_='info')
  nombre_piloto = element.find('h6').text

  labels = info.find('div', class_ = 'labels').text
  carreras = re.findall(r'(\d{1,3}) races', labels)[0]
  cant_pilotos = re.findall(r'(\d{1,3}) drivers', labels)[0]

  try:
    equipo = info.find('p').text.strip()
  except:
    print(f'No se encontro el equipo del anio {anio.strip()}')
    equipo = None
  
  data = {
      'nombre' : nombre_piloto,
      'equipo' : equipo,
      'anio' : int(anio.strip()),
      'carreras' : int(carreras),
      'total pilotos' : int(cant_pilotos),
  }
  
  pilotos.append(data)

import pandas as pd
df = pd.DataFrame(pilotos)
df

No se encontro el equipo del anio 1957
No se encontro el equipo del anio 1956
No se encontro el equipo del anio 1955
No se encontro el equipo del anio 1954
No se encontro el equipo del anio 1953
No se encontro el equipo del anio 1952
No se encontro el equipo del anio 1951
No se encontro el equipo del anio 1950


,nombre,equipo,anio,carreras,total pilotos
0,\n\n\n Max Verstappen\n,Red Bull,2023,23,20
1,\n\n Max Verstappen\n,Red Bull,2022,22,22
2,\n\n Max Verstappen\n,Mercedes,2021,22,21
3,\n\n Lewis Hamilton\n,Mercedes,2020,17,23
4,\n\n Lewis Hamilton\n,Mercedes,2019,21,20
...,...,...,...,...,...
69,\n\n Juan Fangio\n,None,1954,9,97
70,\n\n Alberto Ascari\n,None,1953,9,108
71,\n\n Alberto Ascari\n,None,1952,8,105
72,\n\n Juan Fangio\n,None,1951,8,84


### 3.2) Circuitos historicos
Utilizando beautiful soup se pide descargar los siguientes datos:
- Nombre del circuito
- Ubicacion
- Cantidad de carreras
- Link a la pagina del circuito

Ingresando a la pagina del circuito se debe obtener:
- El piloto con más victorias
- Cantidad de victorias
- El piloto con más Pole positions (piloto mas rapido en la clasificación)
- Cantidad de Pole positions

Nota: es recomendable crear una funcion para scrapear la pagina del circuito y devolver los valores pedidos

In [1]:
f1_circuito_url = 'https://pitwall.app/circuits'


In [7]:
def scrap_row(row):
  registro = row.find('tbody').find('tr')
  nombre = registro.find('td', class_ = 'title').text
  cantidad = registro.find('td', class_='center title').text

  return nombre, cantidad

def scrap_circuito(url):
  base_url = 'https://pitwall.app/'
  response = requests.get(base_url + url)
  soup = BeautifulSoup(response.text, 'html.parser')

  tabla_wins = soup.find_all('div', class_ = 'most-wins')

  try:
    mas_victorias, cant_victorias = scrap_row(tabla_wins[0])
    mas_victorias = mas_victorias.strip()
    cant_victorias = int(cant_victorias)

  except AttributeError:
    print(f'No se encontraron las victorias de {nombre}')
    mas_victorias, cant_victorias = None, None
  try:
    mas_pole, cant_pole = scrap_row(tabla_wins[1])
    mas_pole = mas_pole.strip()
    cant_pole = int(cant_pole)

  except AttributeError:
    print(f'No se encontraron los poles de {nombre}')
    mas_pole, cant_pole = None, None
  return {
    'piloto_vict': mas_victorias,
    'cant_vict': cant_victorias,
    'piloto_pole': mas_pole,
    'cant_pole': cant_pole,   
  }

pistas = []
response = requests.get(f1_circuito_url)
soup = BeautifulSoup(response.text, 'html.parser')

tabla = soup.find('tbody')

for circuito in tabla.find_all('tr'):
  a_tag = circuito.find('a')
  url = a_tag['href']
  nombre = a_tag.text

  ubicacion = circuito.find('td', class_ = 'nowrap').text
  cant_carreras = circuito.find('td', class_ = 'minlg center').text

  data_circuito = scrap_circuito(url)

  data = {
      "nombre" : nombre,
      "ubicacion": ubicacion.strip(),
      "cant_carrearas" : int(cant_carreras),
      **data_circuito
  }
  pistas.append(data)
import pandas as pd

df = pd.DataFrame(pistas)
df

No se encontraron las victorias de Baku City Circuit
No se encontraron los poles de Baku City Circuit
No se encontraron las victorias de Las Vegas Strip Street Circuit
No se encontraron los poles de Las Vegas Strip Street Circuit


,nombre,ubicacion,cant_carrearas,piloto_vict,cant_vict,piloto_pole,cant_pole
0,A1-Ring,"Spielburg, Austria",25,Alain Prost,3.0,Niki Lauda,3.0
1,Adelaide Street Circuit,"Adelaide, Australia",11,Alain Prost,2.0,Ayrton Senna,6.0
2,Ain Diab,"Casablanca, Morocco",1,Stirling Moss,1.0,Mike Hawthorn,1.0
3,Aintree,"Liverpool, UK",5,Stirling Moss,2.0,Stirling Moss,2.0
4,Albert Park Grand Prix Circuit,"Melbourne, Australia",26,Michael Schumacher,4.0,Lewis Hamilton,8.0
...,...,...,...,...,...,...,...
74,Valencia Street Circuit,"Valencia, Spain",5,Sebastian Vettel,2.0,Sebastian Vettel,3.0
75,Watkins Glen,"New York State, USA",20,Jim Clark,3.0,Graham Hill,3.0
76,Yas Marina Circuit,"Abu Dhabi, UAE",15,Lewis Hamilton,5.0,Lewis Hamilton,5.0
77,Zeltweg,"Styria, Austria",1,Lorenzo Bandini,1.0,Graham Hill,1.0
